# Guía de Instalación y Práctica — Sesión 1 (Notebook)
Curso: **IA en el Aula — Nivel Avanzado**  
Profesor: **Luis Daniel Benavides Navarro**  
Fecha: **22 de octubre de 2025**

Este cuaderno guía a los participantes para configurar el entorno, conectarse a una API de modelos de lenguaje y realizar las primeras consultas con **parámetros clave** como `temperature`, `max_tokens`, `top_p`, etc. Se incluyen explicaciones paso a paso y ejercicios.


## 1) Requisitos previos
- Python 3.10 o superior
- Cuenta y **clave API** en un proveedor de modelos de lenguaje (ej. OpenAI)
- Editor/entorno: VSCode, Jupyter o Colab
- Conexión a Internet

Si estás en **Colab**, puedes ejecutar el siguiente comando para instalar dependencias. En local, usa tu terminal.

In [2]:
# (Opcional en local/Colab) Instalar dependencias
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 2) Preparar variables de entorno
Crea un archivo `.env` en la carpeta del proyecto con:

```
OPENAI_API_KEY=tu_clave_aqui
```
Nunca publiques tu clave. Evita subir `.env` a repositorios públicos.


In [3]:
# 3) Cargar la clave y crear el cliente
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


Si está en Google Colab, debe incluir la llave en los secretos (en el menú de la izquierda), activar la variable y luego inicializar así:

In [4]:
# 3) Cargar la clave y crear el cliente en google Colab
from openai import OpenAI
from google.colab import userdata


client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
print("Cliente inicializado. Modelo listo para consultas.")

ModuleNotFoundError: No module named 'google'

## 4) Parámetros importantes de la API (explicación breve)
- **`model`**: identifica el modelo a consultar (p.ej., `gpt-4o-mini`).
- **`messages`**: lista de turnos conversacionales. Usa roles `system` (instrucciones generales), `user` (tu prompt), `assistant` (respuestas previas si las hay).
- **`temperature`**: *aleatoriedad* de la salida. Valores bajos (0.0–0.3) hacen respuestas más deterministas; valores altos (0.7–1.0) hacen respuestas más creativas.
- **`top_p`**: alternativa a `temperature` basada en muestreo por probabilidad acumulada; usa uno u otro (no ambos a la vez a valores lejanos) para afinar el estilo.
- **`max_tokens`**: tope de tokens generados en la respuesta. Si es muy bajo, el texto puede cortarse.
- **`stop`**: secuencias de corte; si aparecen, la generación se detiene.
- **`frequency_penalty` / `presence_penalty`**: penalizaciones para reducir repeticiones y fomentar aparición de nuevos términos.

**Buenas prácticas:**
- Controlar `temperature` (0.2–0.7) según la tarea (evaluación → bajo; lluvia de ideas → medio/alto).
- Estructurar prompts y, cuando sea útil, **pedir JSON** para facilitar la automatización.
- Evitar incluir datos personales o sensibles en prompts.


In [5]:
# 5) Primera consulta: respuesta libre
prompt = "Explica en dos frases qué es la inteligencia artificial en la educación."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7  # más creativo que 0.2, menos que 1.0
)
print(response.choices[0].message.content)

La inteligencia artificial en la educación se refiere al uso de tecnologías que simulan la capacidad humana de aprender y resolver problemas, con el fin de personalizar el aprendizaje y mejorar la eficiencia educativa. Estas herramientas pueden analizar el rendimiento de los estudiantes, adaptar contenidos y proporcionar retroalimentación instantánea, facilitando así un aprendizaje más individualizado y efectivo.


### Comentarios sobre `temperature`
- A `0.0–0.2`: respuestas casi siempre iguales (útil en **rubricas** o **explicaciones estándar**).
- A `0.5–0.8`: más variación léxica/estilística (útil en **generación de materiales** o **brainstorming**).
- A `>0.9`: creatividad alta pero riesgo de salidas menos precisas.


In [ ]:
# 6) Respuesta estructurada en JSON para automatización
import json

query = "¿Qué es aprendizaje supervisado?"
schema_instruction = (
    "Responde en formato JSON con las claves: operation, input, output. "
    "operation debe ser 'explanation'; input debe repetir la pregunta; output la explicación clara y breve."
)
response_json = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": query}
    ],
    temperature=0.3,       # más determinista para formatos estructurados
    max_tokens=300         # suficiente para una explicación breve
)
text = response_json.choices[0].message.content
print(text)

# (Opcional) intentar cargar como JSON si el modelo devolvió un objeto válido
try:
    data = json.loads(text)
    print("\nValid JSON →", data)
except json.JSONDecodeError:
    print("\nLa salida no es JSON válido literal. Puedes parsearla manualmente o usar validadores/funciones JSON del proveedor.")

{
  "operation": "explanation",
  "input": "¿Qué es aprendizaje supervisado?",
  "output": "El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida correcta para nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida correcta para nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión.'}


## 7) Ejercicios propuestos
1. Cambia `temperature` a 0.1, 0.5 y 0.9 y compara el estilo de las respuestas.
2. Pide que el modelo responda **siempre en JSON** usando un `system` que lo exija. Verifica si cumple.
3. Crea un prompt de tu área (p.ej., *programación*, *arquitectura*, *matemáticas*) que devuelva un JSON con `operation`, `input`, `steps` (lista) y `output`.
4. Limita la longitud con `max_tokens` y observa si corta.


In [ ]:
# Plantilla reutilizable para el curso
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.3,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura jerárquica para tomar decisiones basadas en características de los datos. Cada nodo interno representa una prueba sobre una característica, cada rama representa el resultado de esa prueba, y cada hoja representa una clase o resultado final, permitiendo así clasificar o predecir resultados a partir de nuevas entradas."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada del conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida correcta para nuevas entradas. Se utiliza comúnmente en tareas como clasificación y regresión.

In [ ]:
# Punto 1 Cambia `temperature` a 0.1, 0.5 y 0.9 y compara el estilo de las respuestas.
for t in [0.1, 0.5, 0.9]:
 response = client.chat.completions.create(
 model="gpt-4o-mini",
 messages=[{"role": "user", "content": "Explica brevemente el principio de funcionamiento de un árbol de decisión."}],
 temperature=t,
 max_tokens=50
 )
 print(f"--- temperature={t} ---")
 print(response.choices[0].message.content)

--- temperature=0.1 ---
Un árbol de decisión es un modelo de aprendizaje automático utilizado para la clasificación y la regresión. Su funcionamiento se basa en la representación gráfica de decisiones y sus posibles consecuencias, organizadas en forma de un árbol.

1. **Estructura**:
--- temperature=0.5 ---
Un árbol de decisión es un modelo de aprendizaje automático utilizado para clasificación y regresión. Su funcionamiento se basa en dividir un conjunto de datos en subconjuntos más pequeños y homogéneos a través de decisiones secuenciales.

El principio de funcionamiento se
--- temperature=0.9 ---
Un árbol de decisión es una herramienta de modelado predictivo que se utiliza en el campo de la estadística y el aprendizaje automático para tomar decisiones basadas en datos. Su funcionamiento se basa en una estructura jerárquica similar a un árbol,


In [14]:
# Punto 2 Pide que el modelo responda **siempre en JSON** usando un `system` que lo exija. Verifica si cumple.
print(ask_model(
    "Explica brevemente qué es una red neuronal convolucional.",
    system="Responde SIEMPRE en formato JSON con las claves: {'prompt': <texto del usuario>, 'respuesta': <tu respuesta>}"
))

{
  "prompt": "Explica brevemente qué es una red neuronal convolucional.",
  "respuesta": "Una red neuronal convolucional (CNN) es un tipo de red neuronal diseñada para procesar datos con una estructura de cuadrícula, como imágenes. Utiliza capas convolucionales que aplican filtros para detectar características locales, como bordes y texturas, a lo largo de la imagen. Estas redes son especialmente efectivas en tareas de visión por computadora, como clasificación de imágenes y detección de objetos, debido a su capacidad para aprender patrones jerárquicos y reducir la dimensionalidad de los datos."
}


In [20]:
# Punto 3 Crea un prompt de tu área (p.ej., *programación*, *arquitectura*, *matemáticas*) que devuelva un JSON con `operation`, `input`, `steps` (lista) y `output`.

messages = [
    {
        "role": "system",
        "content": (
            "format-json: Responde SIEMPRE en formato JSON con las claves: "
            "'operation', 'input', 'steps' (lista) y 'output'. "
            "Incluye explicaciones matemáticas y código bien comentado."
        )
    },
    {
        "role": "user",
        "content": (
            "Escribe una función en Python que calcule la suma de los primeros n números naturales "
            "y explica detalladamente cómo se deduce la fórmula matemática usada."
        )
    }
]

# Parámetros del modelo
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.5,
    max_tokens=400
)

# Mostrar resultado
print(resp.choices[0].message.content)


```json
{
  "operation": "calculate_sum_of_n_natural_numbers",
  "input": {
    "n": "Número natural hasta el cual se desea calcular la suma"
  },
  "steps": [
    "La suma de los primeros n números naturales se puede expresar como S = 1 + 2 + 3 + ... + n.",
    "Esta suma se puede simplificar usando la fórmula matemática: S = n * (n + 1) / 2.",
    "La fórmula se deduce al observar que si agrupamos los números de la siguiente manera: (1 + n), (2 + (n-1)), (3 + (n-2)), ..., se obtienen n/2 pares, cada uno sumando (n + 1).",
    "Por lo tanto, el número total de pares es n/2, y cada par suma (n + 1).",
    "Multiplicando el número de pares por la suma de cada par, obtenemos: S = (n/2) * (n + 1).",
    "Finalmente, esta fórmula se simplifica a S = n * (n + 1) / 2."
  ],
  "output": {
    "code": "def suma_numeros_naturales(n):\n    # Verificamos que n sea un número natural no negativo\n    if n < 1:\n        return 0  # La suma de números naturales empieza desde 1\n    # Aplicamos la fór

In [ ]:
# Punto 4 Limita la longitud con `max_tokens` y observa si corta; se acorta el token de 400 a 50

def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.3,
              max_tokens: int = 50,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura en forma de árbol para tomar decisiones basadas en características


## 9) Solución de problemas comunes
- **`openai.AuthenticationError` / `401`**: clave inválida o no cargada; revisa tu `.env` y reinicia el kernel.
- **`Rate limit`**: excediste el número de solicitudes por minuto; espera unos segundos y reintenta.
- **`model_not_found`**: el modelo no existe o no tienes acceso; cambia a uno disponible en tu cuenta.
- **Salidas no JSON**: fija `temperature=0.0–0.3`, agrega instrucciones `system` estrictas y/o usa funciones nativas de validación JSON si el proveedor las ofrece.


## 10) Próximos pasos
En la siguiente sesión se verán **consultas avanzadas**, manejo de contexto y el punto de partida para construir un **asistente con RAG** (recuperación de conocimiento) usando materiales del curso.
